# Install Packages

In [ ]:
# %%capture
# !pip install pip3-autoremove
# !pip-autoremove torch torchvision torchaudio -y
# !pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
# !pip install unsloth

# Import Libraries

In [ ]:
import re
import torch
import pandas as pd
from datasets import load_dataset
from trl import SFTTrainer
from tqdm.auto import tqdm
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported, FastLanguageModel
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Load LLAMA 3.2 1B Model and Tokenizer

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None # None for auto detection.
load_in_4bit = True # 4bit quantization to reduce memory usage. 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    #token = "" HF_Token for gated models
)

# Add LoRA Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# Load Original Training Data and Merge with New Data

In [ ]:
label_map = {
    0: "without Adverse Drug Events",
    1: "with Adverse Drug Events"
}


train = pd.read_csv("./data/en_train_data_SMM4H_2025_clean.csv")

train, val = train_test_split(train, test_size=0.2, random_state=20)

train["instruction"] = "Classify this math problem into two topics: with Adverse Drug Events and without. Adverse Drug Events are negative medical side effects associated with a drug"
train["label"] = train["label"].map(label_map)
train = train.rename(columns={"label": "output", "text": "input"})
train.to_csv("train_updated.csv", index=False)


dataset = load_dataset("csv", data_files="train_updated.csv", split="train")

# Prepare Data

In [ ]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
dataset, dataset[0]

In [ ]:
print(dataset[0]["text"])

# Setup Trainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 642,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
    ),
)

# Show current memory stats

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Start Training

In [ ]:
trainer_stats = trainer.train()

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

# Save Model (Just LoRA Adapters) and Tokenzer

In [ ]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

# Load the Saved Model and Tokenizer

In [ ]:
max_seq_length = 2048
dtype = None # None for auto detection.
load_in_4bit = True # 4bit quantization to reduce memory usage. 

# if False:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

# Make Predictions on Comptetion Test Set

In [ ]:
train = pd.read_csv("./data/en_train_data_SMM4H_2025_clean.csv")
_, val = train_test_split(train, test_size=0.2, random_state=20)
public_set = val
public_set

In [ ]:
FastLanguageModel.for_inference(model)

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

public_set["instruction"] = "Classify this math problem into two topics: with Adverse Drug Events and without. Adverse Drug Events are negative medical side effects associated with a drug"
public_set.rename(columns = {"text": "input"}, inplace=True)

raw_outputs = []
for i in tqdm(range(len(public_set))):
  inputs = tokenizer(
  [
      prompt.format(
          public_set.iloc[0]["instruction"], 
          public_set.iloc[i]["input"], 
          "",
      )
  ], return_tensors = "pt", truncation = True, max_length = 2048).to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
  raw_outputs.append(tokenizer.batch_decode(outputs))

In [ ]:
def parse_output(output):
    # re_match = re.search(r'### Response:\n(.*?)<\|end▁of▁sentence\|>', output, re.DOTALL)
    re_match = re.search(r'### Response:\n(.*?)</s>', output, re.DOTALL)
    if re_match:
        response = re_match.group(1).strip()
        return response
    else:
        return ''

In [ ]:
public_set["raw_outputs"] = [raw_output[0] for raw_output in raw_outputs]
print(public_set["raw_outputs"].iloc[1])

In [ ]:
print(parse_output(public_set["raw_outputs"].iloc[1]))

In [ ]:
public_set["parsed_outputs"] = public_set["raw_outputs"].apply(parse_output)
public_set

In [ ]:
label_map = {
    0: "without Adverse Drug Events",
    1: "with Adverse Drug Events"
}

label2id = {v:k for k,v in label_map.items()}
label2id

In [ ]:
public_set["predicted_label"] = public_set["parsed_outputs"].map(label2id)
public_set

# Make Submission to the Competition

In [ ]:
public_set["label"] = public_set["label"].fillna(0).astype(int)
public_set.rename(columns = {"input": "text"})
public_set

In [ ]:
public_set[["Question", "label", "predicted_label"]].to_csv("submission.csv", index=False)
prediction = pd.read_csv("submission.csv")
prediction

In [ ]:
true_labels, prediced_labels = prediction['label'].values, prediction['predicted_label'].values

In [ ]:
from sklearn.metrics import (accuracy_score, 
                             precision_score, 
                             recall_score, 
                             f1_score, 
                             confusion_matrix, 
                             classification_report)

In [ ]:
validation_accuracy = accuracy_score(prediced_labels, true_labels)
validation_precision = precision_score(prediced_labels, true_labels)
validation_recall = recall_score(prediced_labels, true_labels)
validation_f1_micro = f1_score(prediced_labels, true_labels, average='micro')
validation_f1_macro = f1_score(prediced_labels, true_labels, average='macro')

In [ ]:
print(
    f"Accuracy: {validation_accuracy}\n",
    f"Precision: {validation_precision}\n",
    f"Recall: {validation_recall}\n",
    f"F1 micro: {validation_f1_micro}\n",
    f"F1 macro: {validation_f1_macro}\n"
)

In [ ]:
report = classification_report(true_labels, prediced_labels)
print(report)

In [ ]:
cm = confusion_matrix(true_labels, prediced_labels)

In [ ]:
with plt.style.context('default'):  
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', cbar=False,
                xticklabels=["0", "1"], yticklabels=["0", "1"])
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()